In [1]:
import argparse
import glob
from hydra import compose, initialize
import hydra
import numpy as np
import pandas as pd
import torch
import yaml
from omegaconf import DictConfig, OmegaConf
from dl4cv.utils.utils import set_seed
from dl4cv.lightning_classes.plmodel import LitODModel
from dl4cv.datasets import build_taco 
from dl4cv.datasets.taco_data import taco_val_test_collate_fn
from dl4cv.utils.object_detect_utils import get_iou, fix_orientation
from tqdm import tqdm
import os


In [2]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path="./configs")
inference_cfg = compose(config_name="config_taco_training")
inference_cfg["inference"]["run_name"] = 'Resnet50_Whale_250x250'
inference_cfg["inference"]["device"] = 'cuda:1'
path = f"outputs/{inference_cfg.inference.run_name}/.hydra/config.yaml"
with open(path) as cfg:
    cfg_yaml = yaml.safe_load(cfg)
cfg_yaml["inference"] = inference_cfg["inference"]
cfg_yaml["datamodule"]["test"]["params"]["num_to_return"] = 100
cfg = OmegaConf.create(cfg_yaml)


In [3]:
def main(cfg: DictConfig) -> None:
    """
    Run pytorch-lightning model inference
    Args:
        cfg: hydra config
    Returns:
        None
    """
    set_seed(cfg.training.seed)

    device = torch.device(cfg.inference.device)

    model_names = glob.glob(f"outputs/{cfg.inference.run_name}/saved_models/*.ckpt")
    _, _, test_set = build_taco(cfg)
    test_set.num_to_return = 25
    # Dirty trick to get the ground truth boxes
    loader = torch.utils.data.DataLoader(
        test_set,
        collate_fn=taco_val_test_collate_fn,
        batch_size=1,
        num_workers=1,
        shuffle=False,
    )
    lit_model = LitODModel.load_from_checkpoint(checkpoint_path=model_names[0], cfg=cfg)
    lit_model.to(device)
    predictions = []
    for batch in tqdm(loader):
        # move batch elements to device
        batch = tuple(b.to(device) for b in batch)
        predictions.append(
            lit_model.nms_on_image(
                batch,
            )
        )

    return predictions




In [4]:
predictions = main(cfg)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:45<00:00,  5.44it/s]


In [6]:
# find an index in predictions where the kept_preds are not empty
for i in range(len(predictions)):
    if len(predictions[i]['kept_preds']) > 0:
        print(i)

37
53
80
92
106
112
118
123
125
173
177
240


In [ ]:
def mean_average_precision(pred, truth, iou_threshold = 0.5, num_classes = 29, per_class = False):
    # compute mAP https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/object_detection/metrics/mean_avg_precision.py
    # `pred` is given in [[{'bbox':{'x1', 'x2', 'y1', 'y2'}, 'class'(int), 'conf'}, ...], ...]
    # `truth` is given in [[{'x1', 'x2', 'y1', 'y2', 'class'(int)}, more boxes...], ...]
    average_precisions = []

    # used for numerical stability later on
    epsilon = 1e-6

    for c in range(1, num_classes): # class '0' is background

        TP = 0
        FP = 0
        total_true_bboxes = 0

        # list detected(predicted) objects of class 'c'
        detections = []

        for idx, prs in enumerate(pred):
          for pr in prs:
            if pr['class'] == c:
                detections.append((pr['conf'], idx, pr['bbox']))

        # make checkbox for checking whether gt object was detected
        total_true_bboxes = 0
        is_detected = []
        for gts in pred:
          is_detected.append([False for _ in gts])
          total_true_bboxes += sum([gt['class']==c for gt in gts])

        detections.sort(reverse=True)

        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))

        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            # Only take out the ground_truths that have the same
            # training idx as detection
            num_gts = len(truth[detection[1]])

            # find most closest g.t box to pred as best_gt_idx
            best_iou = 0
            for idx, gt in enumerate(truth[detection[1]]):
                #print(gt, detection[2])
                
                iou = get_iou(gt, detection[2])

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            # if considered found
            #try:
            #  print(best_iou, truth[detection[1]][best_gt_idx], detection[2])
            #except:
            #  pass
            if best_iou > iou_threshold:
                # only detect ground truth detection once
                if is_detected[detection[1]][best_gt_idx] == False:
                    # true positive and add this bounding box to seen
                    TP[detection_idx] = 1
                    is_detected[detection[1]][best_gt_idx] = True
                else: # duplicate is FP
                    FP[detection_idx] = 1
            # if IOU is lower then the detection is a false positive
            else:
                FP[detection_idx] = 1
        
        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        #if len(TP)>0 and len(FP)>0:
        #  print(TP_cumsum[-1], FP_cumsum[-1])
        #print(total_true_bboxes)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)  # ratio of detected objects!
        precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)         # ratio of predictions that are true objects!

        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        #print(precisions, recalls, torch.trapz(precisions, recalls))
        average_precisions.append(torch.trapz(precisions, recalls))
        #print('----------')
    if per_class: 
        return average_precisions
    else:
        return sum(average_precisions) / len(average_precisions)

In [10]:
predictions[92]

{'kept_preds': [{'bbox': {'x1': 24, 'y1': 42, 'x2': 233, 'y2': 202},
   'conf': 0.24010595679283142,
   'pred_class': 18,
   'true_class': 12,
   'image_id': 823}],
 'P': [{'bbox': {'x1': 16, 'y1': 41, 'x2': 224, 'y2': 162},
   'conf': 0.18989835679531097,
   'pred_class': 28,
   'true_class': 18,
   'image_id': 823},
  {'bbox': {'x1': 27, 'y1': 45, 'x2': 235, 'y2': 203},
   'conf': 0.2335733324289322,
   'pred_class': 28,
   'true_class': 12,
   'image_id': 823},
  {'bbox': {'x1': 34, 'y1': 54, 'x2': 224, 'y2': 152},
   'conf': 0.25377675890922546,
   'pred_class': 28,
   'true_class': 18,
   'image_id': 823},
  {'bbox': {'x1': 16, 'y1': 41, 'x2': 237, 'y2': 203},
   'conf': 0.22333475947380066,
   'pred_class': 18,
   'true_class': 12,
   'image_id': 823},
  {'bbox': {'x1': 29, 'y1': 45, 'x2': 227, 'y2': 162},
   'conf': 0.23895218968391418,
   'pred_class': 28,
   'true_class': 18,
   'image_id': 823},
  {'bbox': {'x1': 29, 'y1': 45, 'x2': 218, 'y2': 162},
   'conf': 0.2231355607509